<a href="https://colab.research.google.com/github/zubayr1/AtlanTic/blob/master/Task3_Improving_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 3: Challenge Task

##Import Modules

In [ ]:
from google.colab import files
import pandas as pd
import io
import string

import numpy as np
import random

import time
import gc
from google.colab import drive

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from google.colab import drive

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!rm -rf /content/zk_packages

!git clone https://github.com/zubayr1/zk_packages

import zk_packages.modules as modules

Cloning into 'zk_packages'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 44 (delta 17), reused 41 (delta 14), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:
!rm -rf /content/ar_packages

!git clone https://github.com/zubayr1/ar_packages

import ar_packages.sentiments as sentiments

Cloning into 'ar_packages'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [ ]:
pip install pyiwn 

In [ ]:
!pip install gdown==3.6.0

In [ ]:
def load_model(my_file_id):
  !gdown https://drive.google.com/uc?id={my_file_id}

In [ ]:
load_model("1OAZ6puJVlul3mFQbNsXmBxRFweqAatEj") 

Downloading...
From: https://drive.google.com/uc?id=1OAZ6puJVlul3mFQbNsXmBxRFweqAatEj
To: /content/modelnew.pt
139MB [00:02, 57.0MB/s]


In [ ]:
import pyiwn

2021-03-31:09:32:18,514 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2021-03-31:09:32:20,109 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2021-03-31:09:32:21,305 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2021-03-31:09:32:21,314 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data


##Set parameters and hyperparameters

In [ ]:
embedding_size = 300
learning_rate = 0.01

vocabulary_size = 19365 #We have found this vocabulary size for Hindi dataset in task1 and then have set on this size in task1 and task2

epochs = 10

##Set Cuda

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


##Create instance of all models:

In [ ]:
def instance_model(PATH, model):

  checkpoint = torch.load(PATH)
  # if torch.cuda.is_available():
  #   model.load_state_dict(checkpoint['model_state_dict'])
  
  # else:
  model.load_state_dict(checkpoint['model_state_dict'])

  print(model.eval())

  return model


In [ ]:
hindi_embedding_model = modules.Word2vec( vocabulary_size, embedding_size)


In [ ]:
hindi_embedding_model= instance_model("/content/modelnew.pt", hindi_embedding_model)

Word2vec(
  (FC1): Linear(in_features=19365, out_features=300, bias=False)
  (FC2): Linear(in_features=300, out_features=19365, bias=False)
)


## Load Hindi and Bengali datasets

###Hindi dataset

In [ ]:
uploaded = files.upload()

hindi_df = pd.read_csv(io.BytesIO(uploaded['hindi_hatespeech.tsv']),  sep='\t')


Saving hindi_hatespeech.tsv to hindi_hatespeech (1).tsv


###Bengali dataset

In [ ]:
uploaded = files.upload()

bengali_df = pd.read_csv(io.BytesIO(uploaded['bengali_hatespeech.csv']))

Saving bengali_hatespeech.csv to bengali_hatespeech (1).csv


##Split Bengali Dataset:

###Data preperation

###Prepare hindi data

In [ ]:
def data_preperation(data):
  #TODO: implement!

  #remove User names

  data.loc[:, 'text'] =  data.loc[:, 'text'].str.replace('(@\w+.*?)',"")



  #remove Punctuations

  data.loc[:, 'text'] = data.loc[:,'text'].str.replace('[{}]'.format(string.punctuation), "")
  data.loc[:, 'text'] = data.loc[:,'text'].str.replace(r'\+', "") 
  data.loc[:, 'text'] = data.loc[:,'text'].str.replace('।', "") 
  data.loc[:, 'text'] = data.loc[:,'text'].str.replace('…', "")


  #remove Stopwords
  !rm -rf /content/stopwords-hi

  !git clone https://github.com/stopwords-iso/stopwords-hi


  f=open("/content/stopwords-hi/stopwords-hi.txt", "r")

  if f.mode == 'r':
    contents =f.read()

  newcontent = contents.split()



  data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (newcontent)]))

  

  #lowercasing the words

  data['text'] = data['text'].str.lower()


  return data

hindi_df = data_preperation(hindi_df)

Cloning into 'stopwords-hi'...
remote: Enumerating objects: 29, done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (29/29), done.


###Prepare Bengali data

In [ ]:
def data_preperation(data):
  #TODO: implement!



  #remove Punctuations

  data.loc[:, 'sentence'] = data.loc[:,'sentence'].str.replace('[{}]'.format(string.punctuation), "")
  data.loc[:, 'sentence'] = data.loc[:,'sentence'].str.replace(r'\+', "") 
  data.loc[:, 'sentence'] = data.loc[:,'sentence'].str.replace('।', " ")


  #remove Stopwords
  !rm -rf /content/stopwords-bn

  !git clone https://github.com/stopwords-iso/stopwords-bn


  f=open("/content/stopwords-bn/stopwords-bn.txt", "r")

  if f.mode == 'r':
    contents =f.read()

  newcontent = contents.split()



  data['sentence'] = data['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (newcontent)]))

  

  #lowercasing the words

  data['sentence'] = data['sentence'].str.lower()


  return data

bengali_df = data_preperation(bengali_df)

Cloning into 'stopwords-bn'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.


##Split Bengali data

In [ ]:
bengali_df =  sentiments.splitting(bengali_df, 5992, 14007, 1)

In [ ]:
bengali_df

,sentence,hate
0,জাতীয় শুয়োর,1
1,পাপে পুর্ন পাপন চেহারায় প্রকাশ পায় ক্রিকেট ধংস...,0
2,শুনলাম কথাজানতাম পাপন বিসিবিই সম্মেলন থাম্বনেইল,0
3,বাজে কমেন্টস করছো বলছি এক তোদের কারণে বাংলাদেশ...,0
4,পাপনের দুশ,0
...,...,...
8011,সালা আবালে নায়ক হা হা,1
8012,নবীকে গালি গায়ে লাগেনা ক্রিকেট খেলা হারাম খেল...,0
8013,টাকার দালাল হালার পুতহালা,1
8014,বাংলাদেশের মানুষের নিয়ম আন্দোলন আন্দোলনের ফল ...,0


## Data Augmentation

###Method to reset indexing

In [ ]:
def reset_index(data):
  data = data.reset_index()
  if 'index' in data.columns:
    del data['index']
  
  return data

###Method for swapping 2 words

In [ ]:
def swapping_hindiwords(train, count):
  lis = list(train['text'].loc[[count]].str.split())[0]
  length = len(lis)
  if length>0:
    rand1 = random.randint(0, length-1)
    rand2 = random.randint(0, length-1)

    lis[rand1], lis[rand2] = lis[rand2], lis[rand1]
    lis = ' '.join(lis)

    train.at[count,'text']=lis
    print('swapping done')




In [ ]:
def swapping_bengaliwords(train, count):
  lis = list(train['sentence'].loc[[count]].str.split())[0]
  length = len(lis)
  if length>0:
    rand1 = random.randint(0, length-1)
    rand2 = random.randint(0, length-1)

    lis[rand1], lis[rand2] = lis[rand2], lis[rand1]
    lis = ' '.join(lis)

    train.at[count,'sentence']=lis
    print('swapping done')
  



###Method to replace word with its synonym

In [ ]:
def replacing_hindi(train):
  count=0
  iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.HINDI)

  for i in train['text']:
    x= int((len(i.split()))/5)
    t=0
    print('count', count)

    #swapping words
    swapping_hindiwords(train, count)

    #synonym replacement
    for j in range(x):
      rand = random.randint(0, x-1)
      
      word = list(train['text'].loc[[count]].str.split())[0][rand]
      try:
        aam_all_synsets = iwn.synsets(word) 
        newword = aam_all_synsets[0].lemma_names()[0]
        print('replacement done')
        train['text'].loc[[count]].str.replace(word, newword)

      except:
        # print('hindi word not found')
        t+=1
        

    count+=1
  
  return train

In [ ]:
def replacing_bengali(train):
  count=0
  iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.BENGALI)

  for i in train['sentence']:
    x= int((len(i.split()))/5)
    t=0
    print('count', count)

    #swapping words
    swapping_bengaliwords(train, count)
    
    #synonym replacement
    for j in range(x):
      rand = random.randint(0, x-1)
      
      word = list(train['sentence'].loc[[count]].str.split())[0][rand]
      try:
        aam_all_synsets = iwn.synsets(word) 
        newword = aam_all_synsets[0].lemma_names()[0]
        print('replacement done')
        train['sentence'].loc[[count]].str.replace(word, newword)

      except:
        # print('hindi word not found')
        t+=1
        

    count+=1
  
  return train

###Data augmentation method which calls previous methods and also create separate Train set, Validation set and Test set

In [ ]:

def augmentation(df, type):
  train, validation, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.8*len(df)), int(.9*len(df))])
              
  
  train = reset_index(train)
  validation = reset_index(validation)
  test = reset_index(test)


  if type==0:
    train = replacing_hindi(train)
  else:
    train = replacing_bengali(train)
              
  return train, validation, test
              


In [ ]:
hindi_train, hindi_validation, hindi_test = augmentation(hindi_df, 0)

2021-03-31:00:33:15,374 INFO     [iwn.py:43] Loading hindi language synsets...
2021-03-31:00:33:16,141 INFO     [utils.py:157] NumExpr defaulting to 2 threads.


Streaming output truncated to the last 5000 lines.
count 2353
swapping done
replacement done
replacement done
replacement done
count 2354
swapping done
count 2355
swapping done
replacement done
replacement done
replacement done
count 2356
swapping done
replacement done
replacement done
count 2357
swapping done
count 2358
swapping done
count 2359
swapping done
replacement done
replacement done
replacement done
replacement done
replacement done
count 2360
swapping done
replacement done
replacement done
replacement done
replacement done
count 2361
swapping done
replacement done
replacement done
replacement done
replacement done
replacement done
replacement done
replacement done
count 2362
swapping done
replacement done
replacement done
count 2363
swapping done
replacement done
replacement done
replacement done
replacement done
count 2364
swapping done
replacement done
replacement done
replacement done
replacement done
replacement done
count 2365
swapping done
replacement done
replacement 

In [ ]:
frames = [hindi_train, hindi_validation, hindi_test]

In [ ]:
hindi_df  = pd.concat(frames)


In [ ]:
bengali_train, bengali_validation, bengali_test = augmentation(bengali_df, 1)

2021-03-31:00:35:37,317 INFO     [iwn.py:43] Loading bengali language synsets...


Streaming output truncated to the last 5000 lines.
swapping done
count 4341
swapping done
count 4342
swapping done
count 4343
swapping done
count 4344
swapping done
count 4345
swapping done
replacement done
replacement done
count 4346
swapping done
replacement done
replacement done
count 4347
swapping done
replacement done
count 4348
swapping done
count 4349
swapping done
replacement done
count 4350
swapping done
count 4351
swapping done
count 4352
swapping done
count 4353
swapping done
count 4354
swapping done
count 4355
swapping done
replacement done
replacement done
count 4356
swapping done
count 4357
swapping done
count 4358
swapping done
count 4359
swapping done
replacement done
count 4360
swapping done
count 4361
swapping done
count 4362
swapping done
count 4363
swapping done
replacement done
replacement done
replacement done
count 4364
swapping done
count 4365
swapping done
count 4366
swapping done
count 4367
swapping done
count 4368
swapping done
replacement done
replacement do

In [ ]:
frames = [bengali_train, bengali_validation, bengali_test]

In [ ]:
bengali_df  = pd.concat(frames)


##Create wordslist and Vocabulary for bengali data

In [ ]:
BENGALIWORDSINLIST,BENGALIV = modules.building_vocabulary(bengali_df, 'sentence')

##Create wordslist and Vocabulary for Hindi data

In [ ]:
HINDIWORDSINLIST, HINDIV =  modules.building_vocabulary(hindi_df, 'text')

##Now we will do custom datasetting for data

###Method for indexing of word

In [ ]:
def word_to_index(word, V):
  count=0
  for i in V['Values']:
    if i.strip() in word.strip() and word.strip() in i.strip():
      return count
    else:
      count+=1
  return count

###Custom datasetting

In [ ]:
class CustomDataset():
  def __init__(self, df, type):
    self.df = df
    self.type= type

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):

 
    if self.type is 'HINDI':
      sentence = self.df['text'][idx]
      target = self.df['task_1'][idx]
    else:
      sentence = self.df['sentence'][idx]
      target = self.df['hate'][idx]


    paddedstr=""
    if self.type is 'HINDI':
      if len(sentence.split())>=30:
        sentencelist =  sentence.split()[:30]
        sentencestr = ' '.join(sentencelist)
        paddedstr+=sentencestr
      else:
        while len(sentence.split())<30:
          sentence = sentence + ' 0'
        paddedstr+=sentence
    else:
      if len(sentence.split())>=25:
        sentencelist =  sentence.split()[:25]
        sentencestr = ' '.join(sentencelist)
        paddedstr+=sentencestr
      else:
        while len(sentence.split())<25:
          sentence = sentence + ' সত্য'
        paddedstr+=sentence



    paddedlist = paddedstr.split()



    index_list=[]

    
    if self.type is 'HINDI':
      for current in paddedlist:
        current_index = word_to_index(current, HINDIV)
        index_list.append(current_index)
      
      index_list = index_list[:30]
      index_list = torch.tensor(index_list)
    else:
      for current in paddedlist:
        current_index = word_to_index(current, BENGALIV)
        index_list.append(current_index)
      
      index_list = index_list[:30]
      index_list = torch.tensor(index_list)


    hatespeech = 0

    if self.type is 'HINDI':
      if target in 'NOT':
        hatespeech=0.0
      else:
        hatespeech = 1.0
    else:
      if target ==0:
        hatespeech=0.0
      else:
        hatespeech = 1.0


    hatespeech = torch.tensor(hatespeech)




    index_list = index_list.to(device)
    hatespeech = hatespeech.to(device)


    return index_list, hatespeech
  

##Create training, validation and testing loader for Bengali and Hindi data

###Bengali data

In [ ]:
bengalitrain_dataset = CustomDataset(bengali_train, 'BENGALI')
bengalivalidation_dataset = CustomDataset(bengali_validation, 'BENGALI')
bengalitest_dataset = CustomDataset(bengali_test, 'BENGALI')

In [ ]:
bengali_trainloader = DataLoader(bengalitrain_dataset, batch_size= 64, shuffle = False, num_workers=0)
bengali_validationloader = DataLoader(bengalivalidation_dataset, batch_size= 64, shuffle = False, num_workers=0)
bengali_testloader = DataLoader(bengalitest_dataset, batch_size= 64, shuffle = False, num_workers=0)

In [ ]:
count=0
for i, j in bengalitrain_dataset:
  count+=1
  if count==4:
    break
  print(i.shape,  j)


torch.Size([30]) tensor(1., device='cuda:0')
torch.Size([30]) tensor(1., device='cuda:0')
torch.Size([30]) tensor(0., device='cuda:0')


###Hindi data

In [ ]:
hinditrain_dataset = CustomDataset(hindi_train, 'HINDI')
hindivalidation_dataset = CustomDataset(hindi_validation, 'HINDI')
hinditest_dataset = CustomDataset(hindi_test, 'HINDI')

In [ ]:
hindi_trainloader = DataLoader(hinditrain_dataset, batch_size= 64, shuffle = False, num_workers=0)
hindi_validationloader = DataLoader(hindivalidation_dataset, batch_size= 64, shuffle = False, num_workers=0)
hindi_testloader = DataLoader(hinditest_dataset, batch_size= 64, shuffle = False, num_workers=0)

##LSTM Class for new sentiment analysis model

In [ ]:
Embedding = hindi_embedding_model.FC1.weight.T.detach()

In [ ]:
from torch.autograd import  Variable

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim

        # EMbedding
        self.embedding = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_size,  _weight = Embedding)
        
        # RNN
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=0.3)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, 500)

        self.fc1 = nn.Linear(500, 300)

        self.fc2 = nn.Linear(300, 100)

        self.fc3 = nn.Linear(100, 1)

        

        self.BN1 = nn.BatchNorm1d(500)

        self.BN2 = nn.BatchNorm1d(300)

        self.drop1 = nn.Dropout2d(p=0.5)
        self.drop2 = nn.Dropout2d(p=0.8)

    
    def forward(self, x):
        

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        

        # Initialize cell state with zeros
        c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))


        h0 = h0.to(device)
        c0 = c0.to(device)


        embeds = self.embedding(x)

            
        # One time step
        out, (hn,cn) = self.rnn(embeds, (h0,c0))
        out = out.to(device)
        hn = hn.to(device)
        cn = cn.to(device)


        out = self.fc(out[:, -1, :])
        out = self.BN1(out)
        out= F.tanh(out)
        out = self.drop1(out)


        out = self.fc1(out)
        out = self.BN2(out)
        out= F.tanh(out)
        out = self.drop2(out)

        out = self.fc2(out)
        out= F.tanh(out)

        out = self.fc3(out)


        m = nn.Sigmoid()
        output = m(out)

        
        return output

###Initialize some parameters

In [ ]:
# Create RNN
input_dim = 19365    # input dimension
hidden_dim = 100  # hidden layer dimension
layer_dim = 2     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(300, hidden_dim, layer_dim, output_dim)

##Set Optimizer and Loss function

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

criterion = nn.BCELoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

##Train

###First we will train the model on HIndi data

In [ ]:
#train the model..
AVG_LOSS_LIST = []
AVG_TIME = []
def train(train_dl, LENGTH):
    print("Training started")
    lastone = 9999
    
    for epoch in range(epochs):
        start_time = time.time()
        avg_loss = 0
        count = 0

        for xb,yb in train_dl: 

          xb = xb.to(device)
          yb = yb.to(device)

         
          
          # 1. Generate predictions
          optimizer.zero_grad()
          print('New x, label generated')

        
          pred = model(xb)

         
          # 2. Calculate loss
          
          yb = yb.unsqueeze(1)

          loss = criterion(pred, yb)

         
          avg_loss+=loss
          count+=1
          # 3. Compute gradients
          loss.backward()
              
          # 4. Update parameters using gradients
          optimizer.step()
              
          # 5. Reset the gradients to zero
          #optimizer.zero_grad()
        
        

        # Print the progress
        print('avg loss at epoch ', epoch, ' ', avg_loss/count)
        remaining = (time.time() - start_time)
        print("--- %s seconds ---" % remaining)
        AVG_LOSS_LIST.append(avg_loss/count)
        AVG_TIME.append(remaining)

        # Terminating Condition
        if (avg_loss/count)> lastone:
          return
        lastone = (avg_loss/count)

train(hindi_trainloader, len(HINDIV))

print("Training finished")

Training started
New x, label generated


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, labe

###Validation

In [ ]:
#Validate the model..
AVG_LOSS_LIST = []
AVG_TIME = []


def test(test_dl, LENGTH):
    print("Testing started")

    start_time = time.time()
    avg_loss = 0
    count = 0
    hitcount=0
    for xb,yb in test_dl: 

      xb = xb.to(device)
      yb = yb.to(device)

         
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
      # print(xb.shape)
    
      pred = model(xb)

    
      ybcount=-1
      for i in pred:
        
        count+=1
        ybcount+=1
        # print(i, yb[ybcount])
        # print('-----------')
        if i <0.5:
          if torch.eq(yb[ybcount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(yb[ybcount], torch.tensor(1.0)):
            hitcount=hitcount+1



      

      # 2. Calculate loss
      
      yb = yb.unsqueeze(1)

      loss = criterion(pred, yb)


      avg_loss+=loss
      

 

    # Print the progress
    print('avg loss at epoch ', avg_loss/count)
    remaining = (time.time() - start_time)
    print("--- %s seconds ---" % remaining)
    AVG_LOSS_LIST.append(avg_loss/count)
    AVG_TIME.append(remaining)

    print('accuracy:', hitcount/count)


test(hindi_validationloader, len(HINDIV))

print("Testing finished")

Testing started


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


avg loss at epoch  tensor(0.0150, grad_fn=<DivBackward0>)
--- 65.43676495552063 seconds ---
accuracy: 0.7811158798283262
Testing finished


###Testing

In [ ]:
#test the model..
AVG_LOSS_LIST = []
AVG_TIME = []


def test(test_dl, LENGTH):
    print("Testing started")

    start_time = time.time()
    avg_loss = 0
    count = 0
    hitcount=0
    for xb,yb in test_dl: 

      xb = xb.to(device)
      yb = yb.to(device)

         
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
      # print(xb.shape)
    
      pred = model(xb)

    
      ybcount=-1
      for i in pred:
        
        count+=1
        ybcount+=1
        # print(i, yb[ybcount])
        # print('-----------')
        if i <0.5:
          if torch.eq(yb[ybcount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(yb[ybcount], torch.tensor(1.0)):
            hitcount=hitcount+1



      

      # 2. Calculate loss
      
      yb = yb.unsqueeze(1)

      loss = criterion(pred, yb)


      avg_loss+=loss
      

 

    # Print the progress
    print('avg loss at epoch ', avg_loss/count)
    remaining = (time.time() - start_time)
    print("--- %s seconds ---" % remaining)
    AVG_LOSS_LIST.append(avg_loss/count)
    AVG_TIME.append(remaining)

    print('accuracy:', hitcount/count)


test(hindi_testloader, len(HINDIV))

print("Testing finished")

Testing started


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


avg loss at epoch  tensor(0.0154, grad_fn=<DivBackward0>)
--- 62.70032811164856 seconds ---
accuracy: 0.7687366167023555
Testing finished


###Now we will directly test the model on Bengali data

In [ ]:
#test the model..
AVG_LOSS_LIST = []
AVG_TIME = []


def test(test_dl, LENGTH):
    print("Testing started")

    start_time = time.time()
    avg_loss = 0
    count = 0
    hitcount=0
    for xb,yb in test_dl: 

      xb = xb.to(device)
      yb = yb.to(device)

         
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
      # print(xb.shape)
    
      pred = model(xb)

    
      ybcount=-1
      for i in pred:
        
        count+=1
        ybcount+=1
        # print(i, yb[ybcount])
        # print('-----------')
        if i <0.5:
          if torch.eq(yb[ybcount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(yb[ybcount], torch.tensor(1.0)):
            hitcount=hitcount+1



      

      # 2. Calculate loss
      
      yb = yb.unsqueeze(1)

      loss = criterion(pred, yb)


      avg_loss+=loss
      

 

    # Print the progress
    print('avg loss at epoch ', avg_loss/count)
    remaining = (time.time() - start_time)
    print("--- %s seconds ---" % remaining)
    AVG_LOSS_LIST.append(avg_loss/count)
    AVG_TIME.append(remaining)

    print('accuracy:', hitcount/count)


test(bengali_testloader, len(HINDIV))

print("Testing finished")

Testing started


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


avg loss at epoch  tensor(0.0228, grad_fn=<DivBackward0>)
--- 63.955878257751465 seconds ---
accuracy: 0.5031210986267166
Testing finished


##Now we will first train the model with Bengali data and then test it

###Train the model

In [ ]:
#train the model..
AVG_LOSS_LIST = []
AVG_TIME = []
def train(train_dl, LENGTH):
    print("Training started")
    lastone = 9999
    
    for epoch in range(epochs):
        start_time = time.time()
        avg_loss = 0
        count = 0

        for xb,yb in train_dl: 

          xb = xb.to(device)
          yb = yb.to(device)

         
          
          # 1. Generate predictions
          optimizer.zero_grad()
          print('New x, label generated')

        
          pred = model(xb)

         
          # 2. Calculate loss
          
          yb = yb.unsqueeze(1)

          loss = criterion(pred, yb)

         
          avg_loss+=loss
          count+=1
          # 3. Compute gradients
          loss.backward()
              
          # 4. Update parameters using gradients
          optimizer.step()
              
          # 5. Reset the gradients to zero
          #optimizer.zero_grad()
        
        

        # Print the progress
        print('avg loss at epoch ', epoch, ' ', avg_loss/count)
        remaining = (time.time() - start_time)
        print("--- %s seconds ---" % remaining)
        AVG_LOSS_LIST.append(avg_loss/count)
        AVG_TIME.append(remaining)

        # Terminating Condition
        if (avg_loss/count)> lastone:
          return
        lastone = (avg_loss/count)

train(bengali_trainloader, len(BENGALIV))

print("Training finished")

Training started
New x, label generated


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, label generated
New x, labe

###Validate the model

In [ ]:
#Validate the model..
AVG_LOSS_LIST = []
AVG_TIME = []


def test(test_dl, LENGTH):
    print("Testing started")

    start_time = time.time()
    avg_loss = 0
    count = 0
    hitcount=0
    for xb,yb in test_dl: 

      xb = xb.to(device)
      yb = yb.to(device)

         
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
      # print(xb.shape)
    
      pred = model(xb)

    
      ybcount=-1
      for i in pred:
        
        count+=1
        ybcount+=1
        # print(i, yb[ybcount])
        # print('-----------')
        if i <0.5:
          if torch.eq(yb[ybcount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(yb[ybcount], torch.tensor(1.0)):
            hitcount=hitcount+1



      

      # 2. Calculate loss
      
      yb = yb.unsqueeze(1)

      loss = criterion(pred, yb)


      avg_loss+=loss
      

 

    # Print the progress
    print('avg loss at epoch ', avg_loss/count)
    remaining = (time.time() - start_time)
    print("--- %s seconds ---" % remaining)
    AVG_LOSS_LIST.append(avg_loss/count)
    AVG_TIME.append(remaining)

    print('accuracy:', hitcount/count)


test(bengali_validationloader, len(BENGALIV))

print("Testing finished")

Testing started


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


avg loss at epoch  tensor(0.0085, grad_fn=<DivBackward0>)
--- 101.96826815605164 seconds ---
accuracy: 0.8639200998751561
Testing finished


###Test the model

In [ ]:
#test the model..
AVG_LOSS_LIST = []
AVG_TIME = []


def test(test_dl, LENGTH):
    print("Testing started")

    start_time = time.time()
    avg_loss = 0
    count = 0
    hitcount=0
    for xb,yb in test_dl: 

      xb = xb.to(device)
      yb = yb.to(device)

         
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
      # print(xb.shape)
    
      pred = model(xb)

    
      ybcount=-1
      for i in pred:
        
        count+=1
        ybcount+=1
        # print(i, yb[ybcount])
        # print('-----------')
        if i <0.5:
          if torch.eq(yb[ybcount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(yb[ybcount], torch.tensor(1.0)):
            hitcount=hitcount+1



      

      # 2. Calculate loss
      
      yb = yb.unsqueeze(1)

      loss = criterion(pred, yb)


      avg_loss+=loss
      

 

    # Print the progress
    print('avg loss at epoch ', avg_loss/count)
    remaining = (time.time() - start_time)
    print("--- %s seconds ---" % remaining)
    AVG_LOSS_LIST.append(avg_loss/count)
    AVG_TIME.append(remaining)

    print('accuracy:', hitcount/count)


test(bengali_testloader, len(BENGALIV))

print("Testing finished")

Testing started


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


avg loss at epoch  tensor(0.0087, grad_fn=<DivBackward0>)
--- 101.91992402076721 seconds ---
accuracy: 0.8689138576779026
Testing finished


##Save the model

In [ ]:
drive.mount('/drive')
PATH = "final_model.pt"

torch.save({
          'model_state_dict': model.state_dict(),
          }, PATH)

Mounted at /drive
